In [1]:
import pandas as pd
from pathlib import Path
import os
import glob

In [2]:
# path may need to be changed
data_dir = "/Users/NathanBick/Documents/Graduate School/MATH517 - Social Network Analysis/final-proj/"

extension = 'csv'
os.chdir(data_dir)
files = glob.glob('*.{}'.format(extension))
print(files)


['pamela_linkedin_connections.csv', 'nathan_linkedin_connections.csv', 'angela_linkedin_connections.csv']


Read in the CSV data for the different sub-networks

In [7]:
linkedin_data = pd.DataFrame()

for file in files:
    print(data_dir + file)
    tmp = pd.read_csv(data_dir + file,skiprows=3)
    tmp['source_file'] = file
    linkedin_data = linkedin_data.append(tmp)

/Users/NathanBick/Documents/Graduate School/MATH517 - Social Network Analysis/final-proj/pamela_linkedin_connections.csv
/Users/NathanBick/Documents/Graduate School/MATH517 - Social Network Analysis/final-proj/nathan_linkedin_connections.csv
/Users/NathanBick/Documents/Graduate School/MATH517 - Social Network Analysis/final-proj/angela_linkedin_connections.csv


/var/folders/rq/3srb061n1gjddbpd8z6mj84w0000gn/T/ipykernel_87618/3818394218.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linkedin_data = linkedin_data.append(tmp)
/var/folders/rq/3srb061n1gjddbpd8z6mj84w0000gn/T/ipykernel_87618/3818394218.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linkedin_data = linkedin_data.append(tmp)
/var/folders/rq/3srb061n1gjddbpd8z6mj84w0000gn/T/ipykernel_87618/3818394218.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linkedin_data = linkedin_data.append(tmp)


In [10]:
#print(linkedin_data)
linkedin_data.groupby(['source_file']).count()

,First Name,Last Name,Email Address,Company,Position,Connected On
source_file,,,,,,
angela_linkedin_connections.csv,255,255,1,254,254,255
nathan_linkedin_connections.csv,641,641,3,639,639,644
pamela_linkedin_connections.csv,64,64,1,63,63,64


Using the data that we read, create adjacency list for networks using alternate possible definitions of edges:

* linkedin connection - there would be three main nodes (each of the group members) and then all of those connections in the rows
* same employer defines an edge - this requires looping through each name to create each as a node. Then loop through the other 

In [14]:
linkedin_data['Full Name'] = linkedin_data['First Name'] + " " + linkedin_data['Last Name']
nodes = linkedin_data['Full Name'].unique() 
companies = linkedin_data['Company'].unique()
print(nodes)

['Forrest Warner' 'Ron Fireman' 'Abhishek Patnia' 'Miao(Annabella) Miao'
 'yanfeng Lai' 'Jocelyn Huang' 'Vania Todorova' 'Dr. Hernani Costa'
 'Rakshitha Sankineni' 'Prarthana Bhattarai' 'Christopher Harris'
 'Md Arafat Hossain Khan' 'Shiv Rathod' 'Anirudh Venkateshwaran'
 'Purva Shanker' 'Josie (Yichen) Zhang' 'Minh B. Le, PhD, CFA'
 'Saif Abuhashish' 'Adithya Ramanathan' 'Diana Mingels' 'Tarek Lahlou'
 'Neil Acharya' 'Alex Bussan' 'Andrew Stern' 'Ilana Gharib'
 'Daniel Jennings' 'Ronit Gopalani' 'Stephen Haptonstahl' "Stalin D'Souza"
 'Parag Jain' 'Jonathan Gross' 'Oluwatobi Olabiyi' 'Graham Park'
 'Nathan Wolfe' 'Benjamin Fineran' 'Arturo Hernandez' 'Zachary Brown, PhD'
 'Victor Alvarez' 'Andrew Oestreicher' 'Alston Clark' 'Oron Gill Haus'
 'Mathias Menasi' 'Carlton Marshall II' 'Thuc Tran' 'Kunlaya Soiaporn'
 'Oliver Bentham' 'Tate Travaglini' 'Jason Filippou ⌨' 'Adam Psarouthakis'
 'Kwan-yuet Ho' 'Amanda (Mandy) Pearce' 'Benjamin Constantine'
 'Skylar Oke' "Han 'Pamela' Phan" 'Bill

In [19]:
employer_network = pd.DataFrame()

linkedin_data = linkedin_data.reset_index()  # make sure indexes pair with number of rows
# loop through the unqiue node values. Create a row in the network. 
# For each unique node, loop through all the nodes again, see if the two have the same employer. 
# if they do have the same employer, add to the adjacency list
for node1 in nodes:
    for node2 in nodes:
        row

ValueError: cannot insert level_0, already exists